In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
import torchtext.experimental
import torchtext.experimental.vectors
from torchtext.experimental.datasets.raw.text_classification import RawTextIterableDataset
from torchtext.experimental.datasets.text_classification import TextClassificationDataset
from torchtext.experimental.functional import sequential_transforms, vocab_func, totensor

import collections
import random
import time

In [2]:
seed = 1234

torch.manual_seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
raw_train_data, raw_test_data = torchtext.experimental.datasets.raw.IMDB()

In [4]:
def get_train_valid_split(raw_train_data, split_ratio = 0.7):

    raw_train_data = list(raw_train_data)
        
    random.shuffle(raw_train_data)
        
    n_train_examples = int(len(raw_train_data) * split_ratio)
        
    train_data = raw_train_data[:n_train_examples]
    valid_data = raw_train_data[n_train_examples:]
    
    train_data = RawTextIterableDataset(train_data)
    valid_data = RawTextIterableDataset(valid_data)
    
    return train_data, valid_data

In [5]:
raw_train_data, raw_valid_data = get_train_valid_split(raw_train_data)

In [6]:
class Tokenizer:
    def __init__(self, tokenize_fn = 'basic_english', lower = True, max_length = None, sos_token = None):
        
        self.tokenize_fn = torchtext.data.utils.get_tokenizer(tokenize_fn)
        self.lower = lower
        self.max_length = max_length
        self.sos_token = sos_token
        
    def tokenize(self, s):
        
        tokens = self.tokenize_fn(s)
        
        if self.lower:
            tokens = [token.lower() for token in tokens]
            
        if self.sos_token is not None:
            tokens = [self.sos_token] + tokens

        if self.max_length is not None:
            tokens = tokens[:self.max_length]
            
        return tokens

In [7]:
max_length = 250
sos_token = '<sos>'

tokenizer = Tokenizer(max_length = max_length, sos_token = sos_token)

In [8]:
s = "hello world, how are you?"

tokenizer.tokenize(s)

['<sos>', 'hello', 'world', ',', 'how', 'are', 'you', '?']

In [9]:
def build_vocab_from_data(raw_data, tokenizer, **vocab_kwargs):
    
    token_freqs = collections.Counter()
    
    for label, text in raw_data:
        tokens = tokenizer.tokenize(text)
        token_freqs.update(tokens)
                
    vocab = torchtext.vocab.Vocab(token_freqs, **vocab_kwargs)
    
    return vocab

In [10]:
max_size = 25_000

vocab = build_vocab_from_data(raw_train_data, tokenizer, max_size = max_size)

In [11]:
def process_raw_data(raw_data, tokenizer, vocab):
    
    raw_data = [(label, text) for (label, text) in raw_data]

    text_transform = sequential_transforms(tokenizer.tokenize,
                                           vocab_func(vocab),
                                           totensor(dtype=torch.long))
    
    label_transform = sequential_transforms(totensor(dtype=torch.long))

    transforms = (label_transform, text_transform)

    dataset = TextClassificationDataset(raw_data,
                                        vocab,
                                        transforms)
    
    return dataset

In [12]:
train_data = process_raw_data(raw_train_data, tokenizer, vocab)
valid_data = process_raw_data(raw_valid_data, tokenizer, vocab)
test_data = process_raw_data(raw_test_data, tokenizer, vocab)

In [13]:
class Collator:
    def __init__(self, pad_idx, batch_first):
        
        self.pad_idx = pad_idx
        self.batch_first = batch_first
        
    def collate(self, batch):
        
        labels, text = zip(*batch)
        
        labels = torch.LongTensor(labels)

        text = nn.utils.rnn.pad_sequence(text, 
                                         padding_value = self.pad_idx,
                                         batch_first = self.batch_first)
        
        return labels, text

In [14]:
pad_token = '<pad>'
pad_idx = vocab[pad_token]
batch_first = False

collator = Collator(pad_idx, batch_first)

In [15]:
batch_size = 256

train_iterator = torch.utils.data.DataLoader(train_data, 
                                             batch_size, 
                                             shuffle = True, 
                                             collate_fn = collator.collate)

valid_iterator = torch.utils.data.DataLoader(valid_data, 
                                             batch_size, 
                                             shuffle = False, 
                                             collate_fn = collator.collate)

test_iterator = torch.utils.data.DataLoader(test_data, 
                                            batch_size, 
                                            shuffle = False, 
                                            collate_fn = collator.collate)

In [16]:
class Transformer(nn.Module):
    def __init__(self, input_dim, emb_dim, n_heads, hid_dim, n_layers, output_dim, dropout, max_length, pad_idx):
        super().__init__()

        self.tok_embedding = nn.Embedding(input_dim, emb_dim, padding_idx = pad_idx)
        self.pos_embedding = nn.Embedding(max_length, emb_dim)
        transformer_layer = nn.TransformerEncoderLayer(emb_dim, n_heads, hid_dim, activation = 'gelu')
        norm = nn.LayerNorm(emb_dim)
        self.transformer = nn.TransformerEncoder(transformer_layer, n_layers, norm)
        self.fc = nn.Linear(emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        # text = [seq len, batch size]

        seq_len, batch_size = text.shape
        
        pos = torch.arange(0, seq_len).unsqueeze(-1).repeat(1, batch_size).to(text.device)
        
        # pos = [seq len, batch size]
        
        embedded_pos = self.pos_embedding(pos)
        embedded_tok = self.tok_embedding(text)
        
        embedded = self.dropout(embedded_pos + embedded_tok)

        # embedded = [seq len, batch size, emb dim]

        transformed = self.transformer(embedded)

        # transformed = [seq len, batch size, emb dim]

        sos_transformed = transformed[0]

        # sos_transformed = [batch size, emb dim]

        prediction = self.fc(self.dropout(sos_transformed))

        # prediction = [batch size, output dim]

        return prediction

In [17]:
input_dim = len(vocab)
emb_dim = 100
n_heads = 10
hid_dim = 1024
n_layers = 3
output_dim = 2
dropout = 0.5
pad_idx = pad_idx

model = Transformer(input_dim, emb_dim, n_heads, hid_dim, n_layers, output_dim, dropout, max_length, pad_idx)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [19]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,265,774 trainable parameters


In [20]:
for n, p in model.named_parameters():
    print(f'name: {n}, shape: {p.shape}')

name: tok_embedding.weight, shape: torch.Size([25002, 100])
name: pos_embedding.weight, shape: torch.Size([250, 100])
name: transformer.layers.0.self_attn.in_proj_weight, shape: torch.Size([300, 100])
name: transformer.layers.0.self_attn.in_proj_bias, shape: torch.Size([300])
name: transformer.layers.0.self_attn.out_proj.weight, shape: torch.Size([100, 100])
name: transformer.layers.0.self_attn.out_proj.bias, shape: torch.Size([100])
name: transformer.layers.0.linear1.weight, shape: torch.Size([1024, 100])
name: transformer.layers.0.linear1.bias, shape: torch.Size([1024])
name: transformer.layers.0.linear2.weight, shape: torch.Size([100, 1024])
name: transformer.layers.0.linear2.bias, shape: torch.Size([100])
name: transformer.layers.0.norm1.weight, shape: torch.Size([100])
name: transformer.layers.0.norm1.bias, shape: torch.Size([100])
name: transformer.layers.0.norm2.weight, shape: torch.Size([100])
name: transformer.layers.0.norm2.bias, shape: torch.Size([100])
name: transformer.lay

In [21]:
def initialize_parameters(m):
    if isinstance(m, nn.Embedding):
        nn.init.normal_(m.weight, std = 0.02)
    elif isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, std = 0.02)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LayerNorm):
        nn.init.ones_(m.weight)
        nn.init.zeros_(m.bias)

In [22]:
model.apply(initialize_parameters)

Transformer(
  (tok_embedding): Embedding(25002, 100, padding_idx=1)
  (pos_embedding): Embedding(250, 100)
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
      (1): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): _LinearWithBias(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100,

In [23]:
glove = torchtext.experimental.vectors.GloVe(name = '6B',
                                             dim = emb_dim)

In [24]:
def get_pretrained_embedding(initial_embedding, pretrained_vectors, vocab, unk_token):
    
    pretrained_embedding = torch.FloatTensor(initial_embedding.weight.clone()).detach()    
    pretrained_vocab = pretrained_vectors.vectors.get_stoi()
    
    unk_tokens = []
    
    for idx, token in enumerate(vocab.itos):
        if token in pretrained_vocab:
            pretrained_vector = pretrained_vectors[token]
            pretrained_embedding[idx] = pretrained_vector
        else:
            unk_tokens.append(token)
        
    return pretrained_embedding, unk_tokens

In [25]:
unk_token = '<unk>'

pretrained_embedding, unk_tokens = get_pretrained_embedding(model.tok_embedding, glove, vocab, unk_token)

In [26]:
model.tok_embedding.weight.data.copy_(pretrained_embedding)

tensor([[-0.0118,  0.0220, -0.0321,  ...,  0.0011,  0.0252,  0.0027],
        [ 0.0154, -0.0052,  0.0104,  ..., -0.0116,  0.0198, -0.0480],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0288, -0.0316,  0.4083,  ...,  0.6288, -0.5348, -0.8080],
        [ 0.0215,  0.0027, -0.0050,  ..., -0.0036, -0.0102,  0.0206],
        [-0.2612,  0.6821, -0.2295,  ..., -0.5306,  0.0863,  0.4852]])

In [27]:
model.tok_embedding.weight.data[pad_idx] = torch.zeros(emb_dim)

In [28]:
optimizer = optim.Adam(model.parameters())

In [29]:
criterion = nn.CrossEntropyLoss()

In [30]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
model = model.to(device)
criterion = criterion.to(device)

In [32]:
def calculate_accuracy(predictions, labels):
    top_predictions = predictions.argmax(1, keepdim = True)
    correct = top_predictions.eq(labels.view_as(top_predictions)).sum()
    accuracy = correct.float() / labels.shape[0]
    return accuracy

In [33]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for labels, text in iterator:

        labels = labels.to(device)
        text = text.to(device)

        optimizer.zero_grad()
        
        predictions = model(text)
        
        loss = criterion(predictions, labels)
        
        acc = calculate_accuracy(predictions, labels)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [34]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for labels, text in iterator:

            labels = labels.to(device)
            text = text.to(device)
            
            predictions = model(text)
            
            loss = criterion(predictions, labels)
            
            acc = calculate_accuracy(predictions, labels)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [35]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [36]:
n_epochs = 10

best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, device)
    
    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 24s
	Train Loss: 0.652 | Train Acc: 59.58%
	 Val. Loss: 0.492 |  Val. Acc: 77.16%
Epoch: 02 | Epoch Time: 0m 24s
	Train Loss: 0.433 | Train Acc: 80.47%
	 Val. Loss: 0.380 |  Val. Acc: 83.43%
Epoch: 03 | Epoch Time: 0m 24s
	Train Loss: 0.351 | Train Acc: 84.96%
	 Val. Loss: 0.366 |  Val. Acc: 83.79%
Epoch: 04 | Epoch Time: 0m 24s
	Train Loss: 0.301 | Train Acc: 87.46%
	 Val. Loss: 0.323 |  Val. Acc: 86.52%
Epoch: 05 | Epoch Time: 0m 24s
	Train Loss: 0.258 | Train Acc: 89.47%
	 Val. Loss: 0.324 |  Val. Acc: 87.18%
Epoch: 06 | Epoch Time: 0m 24s
	Train Loss: 0.232 | Train Acc: 90.77%
	 Val. Loss: 0.320 |  Val. Acc: 86.84%
Epoch: 07 | Epoch Time: 0m 24s
	Train Loss: 0.200 | Train Acc: 92.21%
	 Val. Loss: 0.439 |  Val. Acc: 82.49%
Epoch: 08 | Epoch Time: 0m 24s
	Train Loss: 0.188 | Train Acc: 92.84%
	 Val. Loss: 0.381 |  Val. Acc: 86.18%
Epoch: 09 | Epoch Time: 0m 24s
	Train Loss: 0.159 | Train Acc: 94.02%
	 Val. Loss: 0.358 |  Val. Acc: 87.33%
Epoch: 10 | Epoch T

In [37]:
model.load_state_dict(torch.load('transformer-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.349 | Test Acc: 85.45%


In [38]:
def predict_sentiment(tokenizer, vocab, model, device, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    indexes = [vocab.stoi[token] for token in tokens]
    tensor = torch.LongTensor(indexes).unsqueeze(-1).to(device)
    prediction = model(tensor)
    probabilities = nn.functional.softmax(prediction, dim = -1)
    pos_probability = probabilities.squeeze()[-1].item()
    return pos_probability

In [39]:
sentence = 'the absolute worst movie of all time.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.0066763292998075485

In [40]:
sentence = 'one of the greatest films i have ever seen in my life.'

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.9929355978965759

In [41]:
sentence = "i thought it was going to be one of the greatest films i have ever seen in my life, \
but it was actually the absolute worst movie of all time."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.021573422476649284

In [42]:
sentence = "i thought it was going to be the absolute worst movie of all time, \
but it was actually one of the greatest films i have ever seen in my life."

predict_sentiment(tokenizer, vocab, model, device, sentence)

0.026321368291974068